# Activity Table Creation Process

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Reads the files required in creating the Activity Table
con = pd.read_excel('Source_File_Updated.xlsx','Contracts')
sps_c = pd.read_excel('Source_File_Updated.xlsx','Sps Contracts')
lp = pd.read_excel('Source_File_Updated.xlsx','Last payments')
crmcon = pd.read_excel('Source_File_Updated.xlsx','CRM contracts')
p_period = pd.read_excel('Source_File_Updated.xlsx','Contracts per PaidPeriod')

In [3]:
#Normalizes the datetime format into an absolute date format. i.e It converts the decimal entries to Zero
#sps_c['activation_start'] = pd.to_datetime(sps_c['activation_start']).dt.normalize()
sps_c['activation_end'] = pd.to_datetime(sps_c['activation_end']).dt.normalize()
lp['activation_end'] = pd.to_datetime(lp['activation_end'], errors = 'coerce').dt.normalize()
con['ContractCreation'] = pd.to_datetime(con['ContractCreation']).dt.normalize()
#lp['payment_date'] = pd.to_datetime(lp['payment_date']).dt.normalize()

In [4]:
#Joins the required columns needed to prepare the Table, using the Sps contracts file as the Primary (A) table 

#sps3 = pd.merge(sps1, sps2[['Contract','activation_end']], how='left', on='Contract')


#first_join = con.merge(sps_c, how='left', on='Contract').merge(lp[['Contract','activation_end']],how='left', on='Contract')
#.merge(crmcon[['id','CRMcontract']],how='left', left_on='Contract', right_on='id')
#.merge(p_period[['Contract','PaidPeriods']],how='left', on='Contract')


#df1.merge(df2,on='name').merge(df3,on='name')

#This is the one that works..............................
first_join = pd.merge(pd.merge(pd.merge(pd.merge(sps_c, con, how='left', on='Contract'),lp[['Contract','payment_date','activation_end']],how='left', on='Contract'),
crmcon[['id','CRMcontract']],how='left', left_on='Contract', right_on='id'),p_period[['Contract','PaidPeriods']],how='left', on='Contract')


#first_join.columns = ['Contract','ReportDate','SaleCH','generation','type','LTOperiod','LTOunits','ContractCreation','SpsId',
                     #'Activation_Start', 'Activation_End','Reference_Activation_End','id','CRMcontract','PaidPeriods']


#first_join

#pd.merge(pd.merge(df1,df2,on='name'),df3,on='name')


In [5]:
##Rename the required Date columns respectively for better identification
first_join = first_join.rename(columns = {'activation_start': 'Sps_Activation_Start', 'activation_end_x': 'Sps_Activation_End', 'payment_date': 'LastPayment_Date',
                                          'activation_end_y': 'LastPayment_Activation_End'}, inplace = False)



#first_join["max_activation_end"] = first_join[["Activation_End","Reference_Activation_End"]].max(axis=1)

#first_join

In [6]:
#Create an extra date column that picks the maximium date between the Sps Activation End & Last Payment activation end
first_join["Reference_Activation_End"] = first_join[["Sps_Activation_End","LastPayment_Activation_End"]].max(axis=1)
#first_join.head()

In [7]:
#Converts the column headers from small caps & init caps into full caps

first_join.columns = ['contract','sps_id','sps_activation_start','sps_activation_end','report_date','sales_channel','generation','type',
                       'lto_period','lto_units','contractcreation','lastpayment_date','lastpayment_activation_end','id','crmcontract',
                       'paid_periods','reference_activation_end']


#Creates another table that selects only the required columns from the existing table and arranges the columns in desired order 

second_join = first_join[['report_date','contract','contractcreation','sales_channel','generation','sps_id','crmcontract',
                          'type','lto_period','lto_units','sps_activation_start','reference_activation_end','lastpayment_date','paid_periods']]



#second_join

In [8]:
#Creates a new column that calculates Outage Days as Activation End Date - Reference Date
#It first of all passes today's date into a variable named to_day and then iterates through the whole REFERENCE_ACTIVATION_END
#column to get the days difference between the Activation End Date & Reference Date
import datetime as date
from datetime import date
from datetime import timedelta

today = pd.to_datetime('today').normalize()

to_day = today - timedelta(days=9)

second_join['outage_days'] = (second_join['reference_activation_end'] - to_day).dt.days


#second_join

<ipython-input-8-2b6ea11ca86c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_join['outage_days'] = (second_join['reference_activation_end'] - to_day).dt.days


In [9]:
#Converting specific column(s) datatypes

second_join[['crmcontract','lto_period','paid_periods','outage_days']] = second_join[['crmcontract','lto_period','paid_periods','outage_days']].apply(np.int64)

#second_join.info()

C:\Users\olawale.falodun\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
#Creates an extra column to indicate the active status of a contract by manipulating the Outage Days 
second_join["old_active_status"] = np.where(
   (second_join.outage_days > -60), 
   "Active", 
   "Churn"
    )

#second_join.head()




<ipython-input-10-11295ec57881>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_join["old_active_status"] = np.where(


In [11]:
#Creates an extra column to indicate the LIGHT status of a contract by manipulating the Outage Days
second_join["light_status"] = np.where(
   (second_join.outage_days >= 0),
   "InLight", 
   "InDark"
    )

#second_join.head()


In [12]:
#Churn Check
#second_join[second_join['OUTAGE_DAYS'] == -60]

#second_join[second_join['STATUS'] == 'Retrieval']

In [13]:
#Creates an extra column to indicate the STATUS of a contract by manipulating the Outage Days
second_join['status'] = (
    np.where(
        second_join['outage_days'] > -11, 
        'Paid', 
        np.where(second_join['outage_days'] < -40, 
                 'Retrieval',
                np.where(second_join['outage_days'] < -10, 
                        'Recovery', 'Unavailable'))))

#second_join.head()

In [14]:
#Creating a column that Manipulates the identification of LCP and Customer Category

filters = [
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'LEGACY') & (second_join.type == 'LTO') & (second_join.lto_period == 1800),
   (second_join.sales_channel == 'AIRTEL_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'OUTRIGHT') & (second_join.lto_period == 1),
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'LEGACY') & (second_join.type == 'LTO') & (second_join.lto_period == 1500),
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'LTO') & (second_join.lto_period == 48),
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'LTO') & (second_join.lto_period == 1800),
   (second_join.sales_channel == 'AIRTEL_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'LTO') & (second_join.lto_period == 24),
   (second_join.sales_channel == 'AIRTEL_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'LTO') & (second_join.lto_period == 12),
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'UNIFIED') & (second_join.type == 'OUTRIGHT') & (second_join.lto_period == 1),
   (second_join.sales_channel == 'MTN_NG') & (second_join.generation == 'LEGACY') & (second_join.type == 'LTO') & (second_join.lto_period == -1),
   (second_join.lto_period == second_join.paid_periods)
]
values = ['MTN-LEGACY','OWNER','MTN-LEGACY','MTN-UNIFIED','MTN-UNIFIED','LCP','LCP','OWNER','OWNER','OWNER']

second_join["customer_category"] = np.select(filters, values, default='AIRTEL-UNIFIED')

#second_join.head()

In [15]:
second_join['lastpayment_date'] = second_join['lastpayment_date'].fillna('1900-01-01')
second_join['sps_activation_start'] = second_join['sps_activation_start'].fillna('1900-01-01')
second_join['reference_activation_end'] = second_join['reference_activation_end'].fillna('1900-01-01')
second_join['report_date'] = second_join['report_date'].fillna('1900-01-01')
second_join['type'] = second_join['type'].fillna('null')
second_join['lto_units'] = second_join['lto_units'].fillna('null')
second_join['sales_channel'] = second_join['sales_channel'].fillna('null')
second_join['contractcreation'] = second_join['contractcreation'].fillna('1900-01-01')
second_join['generation'] = second_join['generation'].fillna('null')

In [16]:
#This helps to replace wrong state entries with the correct ones
#ng_location = pd.read_csv('NG_Location_File.csv')

###This is to replace entries for a single column
#ng_location["Customer: Billing State/Province"].replace({"Abuja": "FCT", "Bornu": "Borno", "Cross River ": "Cross River", "Ebonyi ": "Ebonyi"}, inplace=True)


##This is to replace for the entire Data Frame
#ng_location = ng_location.replace(['Abuja','Bornu','Cross River ','Ebonyi ','Federal Capital Territory','Federal Capital Territory (FCT)','Akiti'],['FCT','Borno','Cross River','Ebonyi','FCT','FCT','Ekiti'])

#ng_location.head()

In [17]:

ng_location = pd.read_csv('NG_Location_File.csv')
ng_location = ng_location.replace(['Abuja','Bornu','Cross River ','Ebonyi ','Federal Capital Territory','Federal Capital Territory (FCT)','Akiti'],['FCT','Borno','Cross River','Ebonyi','FCT','FCT','Ekiti'])
ng_location['State'] = ng_location['State'].fillna(ng_location['Customer: Billing State/Province'])
ng_location['State'] = ng_location['State'].fillna(ng_location['Mailing State/Province'])
ng_location['State'] = ng_location['State'].fillna('Unknown')
ng_location['Local Government'] = ng_location['Local Government'].fillna('null')
ng_location['Location (Longitude)'] = ng_location['Location (Longitude)'].fillna(0)
ng_location['Location (Latitude)'] = ng_location['Location (Latitude)'].fillna(0)

state_region = pd.read_excel("State_Region.xlsx")

ng_location_prior = pd.merge(ng_location, state_region, how='left', on='State')

ng_location_complete = ng_location_prior[["Full SFID","Contract Id","Local Government","State","Region","Location (Longitude)","Location (Latitude)"]]

ng_location_complete = ng_location_complete.rename(columns = {'Full SFID': 'contract'}, inplace = False)

second_join = pd.merge(second_join, ng_location_complete[['contract','Local Government','State','Region','Location (Longitude)','Location (Latitude)']], how='left', on='contract')

In [25]:
second_join['State'] = second_join['State'].fillna('Unknown')
second_join['Local Government'] = second_join['Local Government'].fillna('Unknown')
second_join['Location (Longitude)'] = second_join['Location (Longitude)'].fillna(0)
second_join['Location (Latitude)'] = second_join['Location (Latitude)'].fillna(0)
second_join['Region'] = second_join['Region'].fillna('Unknown')

In [29]:
#Filtering with single column
#second_join[(second_join['OUTAGE_DAYS'] == -3) | (second_join['OUTAGE_DAYS'] == -18) | (second_join['OUTAGE_DAYS'] == -34)]

#second_join[second_join['OUTAGE_DAYS'] == -34]

#second_join.isnull().sum().sort_values(ascending = False)
second_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164140 entries, 0 to 164139
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   report_date               164140 non-null  object 
 1   contract                  164140 non-null  object 
 2   contractcreation          164140 non-null  object 
 3   sales_channel             164140 non-null  object 
 4   generation                164140 non-null  object 
 5   sps_id                    164140 non-null  int64  
 6   crmcontract               164140 non-null  int64  
 7   type                      164140 non-null  object 
 8   lto_period                164140 non-null  int64  
 9   lto_units                 164140 non-null  object 
 10  sps_activation_start      164140 non-null  object 
 11  reference_activation_end  164140 non-null  object 
 12  lastpayment_date          164140 non-null  object 
 13  paid_periods              164140 non-null  i

In [32]:
import mysql.connector as msql
from mysql.connector import Error

try:
    conn = msql.connect(host='localhost', 
                           database='lumos_bi_ng', user='root', 
                           password='', port='3308')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to lumos_bi_ng database: ", record)
        #cursor.execute('DROP TABLE IF EXISTS activity;')
        #cursor.execute("TRUNCATE TABLE activity")
        #print('Activity Table Truncated....')
        #print('Re-Creating Activity Table....')
        #cursor.execute("CREATE TABLE activity (report_date DATE NULL,contract VARCHAR(30) NULL,contractcreation DATE NULL,sales_channel VARCHAR(20) NULL,generation VARCHAR(20) NULL,sps_id INT NULL,crmcontract BIGINT NULL,type VARCHAR(20) NULL,lto_period BIGINT NULL,lto_units VARCHAR(20) NULL,sps_activation_start DATETIME NULL,reference_activation_end DATE NULL,lastpayment_date DATETIME NULL,paid_periods BIGINT NULL,outage_days BIGINT NULL,old_active_status VARCHAR(20) NULL,light_status VARCHAR(20) NULL,status VARCHAR(20) NULL)")
        print("inserting into activity....")
        for i,row in second_join.iterrows():
            sql = "INSERT INTO lumos_bi_ng.activity VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            #print("Record inserted")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)
    
print ("Activity Table completely updated")

You're connected to lumos_bi_ng database:  ('lumos_bi_ng',)
inserting into activity....
Activity Table completely updated


In [31]:
#cursor.execute('ALTER TABLE lumos_bi_ng.activity ADD COLUMN customer_category VARCHAR(45) NULL AFTER status')

ProgrammingError: 1060 (42S21): Duplicate column name 'customer_category'